In [2]:
# 因为backprop是不完美的抽象，我们需要一步步的手动实现来理解它，便于debug和优化神经网络
import random
import torch
import torch.nn.functional as F

words = open('names.txt', 'r').read().splitlines()
chars = sorted(list(set(''.join(words))))
s_to_i = {s: i + 1 for i, s in enumerate(chars)}
i_to_s = {i + 1: s for i, s in enumerate(chars)}
s_to_i['.'] = 0
i_to_s[0] = '.'
block_size = 3
vocab_size = len(i_to_s)
print(f"vocab_size: {vocab_size}")


def build_dataset(words):
    X, Y = [], []
    for w in words:
        context = [0] * block_size
        for ch in w + '.':
            ix = s_to_i[ch]
            X.append(context)
            Y.append(ix)
            context = context[1:] + [ix]

    X = torch.tensor(X)
    Y = torch.tensor(Y)
    print(f"x.shape is {X.shape}, y.shape is {Y.shape}")
    return X, Y


random.seed(42)
random.shuffle(words)
n1 = int(0.8 * len(words))
n2 = int(0.9 * len(words))
Xtrain, Ytrain = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xtest, Ytest = build_dataset(words[n2:])

vocab_size: 27
x.shape is torch.Size([182625, 3]), y.shape is torch.Size([182625])
x.shape is torch.Size([22655, 3]), y.shape is torch.Size([22655])
x.shape is torch.Size([22866, 3]), y.shape is torch.Size([22866])


In [3]:
# 比较自己算的导数和pytorch的导数的差异， item是某一项， dt是自己算的
def cmp(item, dt, t):
    # 是否完全一样
    exact = torch.all(dt == t.grad).item()
    # 是否非常接近
    approximate = torch.allclose(dt, t.grad)
    # 最大的差异项是多少
    maxdiff = (dt - t.grad).abs().max().item()
    print(f"{item:15s} | exact: {str(exact):5s}, approximate: {str(approximate):5s}, maxdiff: {maxdiff}")

In [37]:
n_embd = 10
n_hidden = 64

# 初始化参数时，并没有将有的参数设为0，而是乘一个很小的系数，避免手动计算backprop时参数是否为0造成影响
g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size, n_embd), generator=g)
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5 / 3) / ((n_embd * block_size) ** 0.5)
b1 = torch.randn(n_hidden, generator=g) * 0.1
W2 = torch.randn((n_hidden, vocab_size), generator=g) * 0.1
b2 = torch.randn(vocab_size, generator=g) * 0.1

bngain = torch.ones((1, n_hidden)) * 0.1 + 1.0
bnbias = torch.zeros((1, n_hidden)) * 0.1

params = [C, W1, b1, W2, b2, bngain, bnbias]
for p in params:
    p.requires_grad = True
print(sum(p.nelement() for p in params))

4137


In [38]:
batch_size = 32
n = batch_size  # a shorter variable also, for convenience
# construct a minibatch
ix = torch.randint(0, Xtrain.shape[0], (batch_size,), generator=g)
Xb, Yb = Xtrain[ix], Ytrain[ix]

In [49]:
# forward, 为了算导数方便，每一步都是最小化的公式，cross_entropy也需要自己写
emb = C[Xb]  # embed the characters into vectors
embcat = emb.view(emb.shape[0], -1)  # concatenate the vectors
# Linear layer 1
hprebn = embcat @ W1 + b1  # hidden layer pre-activation
# BatchNorm layer
bnmeani = 1 / n * hprebn.sum(0, keepdim=True)
bndiff = hprebn - bnmeani
bndiff2 = bndiff ** 2
bnvar = 1 / (n - 1) * bndiff2.sum(0, keepdim=True)
bnvar_inv = (bnvar + 1e-5) ** -0.5
bnraw = bndiff * bnvar_inv
hpreact = bngain * bnraw + bnbias
# non-linear layer
h = torch.tanh(hpreact)
# Linear layer 1
logits = h @ W2 + b2
# cross_entropy loss
logit_max = logits.max(1, keepdim=True).values
norm_logits = logits - logit_max  # subtract max for numerical stability
counts = norm_logits.exp()
counts_sum = counts.sum(1, keepdim=True)
# x**-1 == 1/x, but sth strange for 1/x here
counts_sum_inv = counts_sum ** -1
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n), Yb].mean()

# PyTorch backward pass
for p in params:
    p.grad = None
for t in [logprobs, probs, counts, counts_sum, counts_sum_inv,  # afaik there is no cleaner way
          norm_logits, logit_max, logits, h, hprebn, bnraw,
          bnvar_inv, bnvar, bndiff2, bndiff, hpreact, bnmeani,
          embcat, emb]:
    t.retain_grad()
loss.backward()
loss

tensor(3.3482, grad_fn=<NegBackward0>)

In [47]:
dlogprobs = torch.zeros_like(logprobs)
dlogprobs[range(n), Yb] = -1.0 / n
cmp("logprobs", dlogprobs, logprobs)

# y = x.log(), so dx=1/x, with chain rule,dprobs/dloss = dx * dlogprobs
dprobs = (1.0 / probs) * dlogprobs
cmp("probs", dprobs, probs)

# counts_sum.shape == (32, 27), counts.shape == (32, 1)
# counts is broadcast to (32, 27) while counts * counts_sum_inv
dcounts_sum_inv = (counts * dprobs).sum(1, keepdim=True)
cmp("counts_sum_inv", dcounts_sum_inv, counts_sum_inv)

# y=x**-1, dx= -1/x**2
dcounts_sum = -1 / counts_sum ** 2 * dcounts_sum_inv
cmp("counts_sum", dcounts_sum, counts_sum)

# counts被用了两次，probs = counts * counts_sum_inv 和 counts_sum_inv = counts.sum(1, keepdim=True) ** -1
dcounts = counts_sum_inv * dprobs + torch.ones_like(counts) * dcounts_sum
cmp("counts", dcounts, counts)

# y = x.exp(), dx=y
dnorm_logits = counts * dcounts
cmp("norm_logits", dnorm_logits, norm_logits)

dlogit_max = (-dnorm_logits).sum(1, keepdim=True)
cmp("logits_max", dlogit_max, logit_max)

# 同样logits也属于两个分支，logit_max是取每列最大的值，所以其导数应该是找到它们的位置放成1
dlogits = dnorm_logits + F.one_hot(logits.max(1).indices, num_classes=logits.shape[1]) * dlogit_max
cmp("logits", dlogits, logits)

# 小技巧，仅通过计算最终导数的shape得到导数的公式：dh.shape == h.shape == (32,64), dlogits.shape == (32, 27), W.shape == (64, 27)
dh = dlogits @ W2.T
cmp("h", dh, h)
dW2 = h.T @ dlogits
cmp("W2", dW2, W2)
db2 = dlogits.sum(0)
cmp("b2", db2, b2)

dhpreact = (1.0 - h ** 2) * dh
cmp("hpreact", dhpreact, hpreact)

dbngain = (bnraw * dhpreact).sum(0, keepdim=True)
cmp("bngain", dbngain, bngain)
dbnraw = bngain * dhpreact
cmp("bnraw", dbnraw, bnraw)
dbnbias = dhpreact.sum(0, keepdim=True)
cmp("bnbias", dbnbias, bnbias)

dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
cmp("bnvar_inv", dbnvar_inv, bnvar_inv)
dbnvar = (-0.5 * (bnvar + 1e-5) ** -1.5) * dbnvar_inv
cmp("bnvar", dbnvar, bnvar)
dbndiff2 = (1.0 / (n - 1)) * torch.ones_like(bndiff2) * dbnvar
cmp("bndiff2", dbndiff2, bndiff2)
dbndiff = bnvar_inv * dbnraw + 2 * bndiff * dbndiff2
cmp("bndiff", dbndiff, bndiff)
dbnmeani = (-dbndiff).sum(0)
cmp("bnmeani", dbnmeani, bnmeani)
dhprebn = dbndiff.clone() + 1.0 / n * (torch.ones_like(hprebn) * dbnmeani)
cmp("hprebn", dhprebn, hprebn)
demcat = dhprebn @ W1.T
cmp("embcat", demcat, embcat)
dW1 = embcat.T @ dhprebn
cmp("W1", dW1, W1)
db1 = dhprebn.sum(0)
cmp("b1", db1, b1)
demb = demcat.view(emb.shape)
cmp("emb", demb, emb)
dC = torch.zeros_like(C)
for k in range(Xb.shape[0]):
    for j in range(Xb.shape[1]):
        i = Xb[k, j]
        dC[i] += demb[k, j]
cmp("C", dC, C)

logprobs        | exact: True , approximate: True , maxdiff: 0.0
probs           | exact: True , approximate: True , maxdiff: 0.0
counts_sum_inv  | exact: True , approximate: True , maxdiff: 0.0
counts_sum      | exact: True , approximate: True , maxdiff: 0.0
counts          | exact: True , approximate: True , maxdiff: 0.0
norm_logits     | exact: True , approximate: True , maxdiff: 0.0
logits_max      | exact: True , approximate: True , maxdiff: 0.0
logits          | exact: True , approximate: True , maxdiff: 0.0
h               | exact: True , approximate: True , maxdiff: 0.0
W2              | exact: True , approximate: True , maxdiff: 0.0
b2              | exact: True , approximate: True , maxdiff: 0.0
hpreact         | exact: True , approximate: True , maxdiff: 0.0
bngain          | exact: True , approximate: True , maxdiff: 0.0
bnraw           | exact: True , approximate: True , maxdiff: 0.0
bnbias          | exact: True , approximate: True , maxdiff: 0.0
bnvar_inv       | exact: 

In [50]:
# Exercise 2: backprop through cross_entropy but all in one go
# to complete this challenge look at the mathematical expression of the loss,
# take the derivative, simplify the expression, and just write it out

# forward pass

# before:
# logit_maxes = logits.max(1, keepdim=True).values
# norm_logits = logits - logit_maxes # subtract max for numerical stability
# counts = norm_logits.exp()
# counts_sum = counts.sum(1, keepdims=True)
# counts_sum_inv = counts_sum**-1 # if I use (1.0 / counts_sum) instead then I can't get backprop to be bit exact...
# probs = counts * counts_sum_inv
# logprobs = probs.log()
# loss = -logprobs[range(n), Yb].mean()

# now:
loss_fast = F.cross_entropy(logits, Yb)
print(loss_fast.item(), 'diff:', (loss_fast - loss).item())

3.348198175430298 diff: 0.0


In [51]:
# backward pass

dlogits = F.softmax(logits, 1)
dlogits[range(n), Yb] -= 1
dlogits /= n

cmp('logits', dlogits, logits) # I can only get approximate to be true, my maxdiff is 6e-9

logits          | exact: False, approximate: True , maxdiff: 6.28642737865448e-09


In [52]:
# Exercise 3: backprop through batchnorm but all in one go
# to complete this challenge look at the mathematical expression of the output of batchnorm,
# take the derivative w.r.t. its input, simplify the expression, and just write it out

# forward pass

# before:
# bnmeani = 1/n*hprebn.sum(0, keepdim=True)
# bndiff = hprebn - bnmeani
# bndiff2 = bndiff**2
# bnvar = 1/(n-1)*(bndiff2).sum(0, keepdim=True) # note: Bessel's correction (dividing by n-1, not n)
# bnvar_inv = (bnvar + 1e-5)**-0.5
# bnraw = bndiff * bnvar_inv
# hpreact = bngain * bnraw + bnbias

# now:
hpreact_fast = bngain * (hprebn - hprebn.mean(0, keepdim=True)) / torch.sqrt(hprebn.var(0, keepdim=True, unbiased=True) + 1e-5) + bnbias
print('max diff:', (hpreact_fast - hpreact).abs().max())

max diff: tensor(4.7684e-07, grad_fn=<MaxBackward1>)


In [53]:
# backward pass

# before we had:
# dbnraw = bngain * dhpreact
# dbndiff = bnvar_inv * dbnraw
# dbnvar_inv = (bndiff * dbnraw).sum(0, keepdim=True)
# dbnvar = (-0.5*(bnvar + 1e-5)**-1.5) * dbnvar_inv
# dbndiff2 = (1.0/(n-1))*torch.ones_like(bndiff2) * dbnvar
# dbndiff += (2*bndiff) * dbndiff2
# dhprebn = dbndiff.clone()
# dbnmeani = (-dbndiff).sum(0)
# dhprebn += 1.0/n * (torch.ones_like(hprebn) * dbnmeani)

# calculate dhprebn given dhpreact (i.e. backprop through the batchnorm)
# (you'll also need to use some of the variables from the forward pass up above)

dhprebn = bngain*bnvar_inv/n * (n*dhpreact - dhpreact.sum(0) - n/(n-1)*bnraw*(dhpreact*bnraw).sum(0))

cmp('hprebn', dhprebn, hprebn) # I can only get approximate to be true, my maxdiff is 9e-10

hprebn          | exact: False, approximate: True , maxdiff: 9.313225746154785e-10


In [55]:
# Exercise 4: putting it all together!
# Train the MLP neural net with your own backward pass

# init
n_embd = 10 # the dimensionality of the character embedding vectors
n_hidden = 200 # the number of neurons in the hidden layer of the MLP

g = torch.Generator().manual_seed(2147483647) # for reproducibility
C  = torch.randn((vocab_size, n_embd),            generator=g)
# Layer 1
W1 = torch.randn((n_embd * block_size, n_hidden), generator=g) * (5/3)/((n_embd * block_size)**0.5)
b1 = torch.randn(n_hidden,                        generator=g) * 0.1
# Layer 2
W2 = torch.randn((n_hidden, vocab_size),          generator=g) * 0.1
b2 = torch.randn(vocab_size,                      generator=g) * 0.1
# BatchNorm parameters
bngain = torch.randn((1, n_hidden))*0.1 + 1.0
bnbias = torch.randn((1, n_hidden))*0.1

parameters = [C, W1, b1, W2, b2, bngain, bnbias]
print(sum(p.nelement() for p in parameters)) # number of parameters in total
for p in parameters:
  p.requires_grad = True

# same optimization as last time
max_steps = 200000
batch_size = 32
n = batch_size # convenience
lossi = []

# use this context manager for efficiency once your backward pass is written (TODO)
with torch.no_grad():

  # kick off optimization
  for i in range(max_steps):

    # minibatch construct
    ix = torch.randint(0, Xtrain.shape[0], (batch_size,), generator=g)
    Xb, Yb = Xtrain[ix], Ytrain[ix] # batch X,Y

    # forward pass
    emb = C[Xb] # embed the characters into vectors
    embcat = emb.view(emb.shape[0], -1) # concatenate the vectors
    # Linear layer
    hprebn = embcat @ W1 + b1 # hidden layer pre-activation
    # BatchNorm layer
    # -------------------------------------------------------------
    bnmean = hprebn.mean(0, keepdim=True)
    bnvar = hprebn.var(0, keepdim=True, unbiased=True)
    bnvar_inv = (bnvar + 1e-5)**-0.5
    bnraw = (hprebn - bnmean) * bnvar_inv
    hpreact = bngain * bnraw + bnbias
    # -------------------------------------------------------------
    # Non-linearity
    h = torch.tanh(hpreact) # hidden layer
    logits = h @ W2 + b2 # output layer
    loss = F.cross_entropy(logits, Yb) # loss function

    # backward pass
    for p in parameters:
      p.grad = None
    #loss.backward() # use this for correctness comparisons, delete it later!

    # manual backprop! #swole_doge_meme
    # -----------------
    dlogits = F.softmax(logits, 1)
    dlogits[range(n), Yb] -= 1
    dlogits /= n
    # 2nd layer backprop
    dh = dlogits @ W2.T
    dW2 = h.T @ dlogits
    db2 = dlogits.sum(0)
    # tanh
    dhpreact = (1.0 - h**2) * dh
    # batchnorm backprop
    dbngain = (bnraw * dhpreact).sum(0, keepdim=True)
    dbnbias = dhpreact.sum(0, keepdim=True)
    dhprebn = bngain*bnvar_inv/n * (n*dhpreact - dhpreact.sum(0) - n/(n-1)*bnraw*(dhpreact*bnraw).sum(0))
    # 1st layer
    dembcat = dhprebn @ W1.T
    dW1 = embcat.T @ dhprebn
    db1 = dhprebn.sum(0)
    # embedding
    demb = dembcat.view(emb.shape)
    dC = torch.zeros_like(C)
    for k in range(Xb.shape[0]):
      for j in range(Xb.shape[1]):
        ix = Xb[k,j]
        dC[ix] += demb[k,j]
    grads = [dC, dW1, db1, dW2, db2, dbngain, dbnbias]
    # -----------------

    # update
    lr = 0.1 if i < 100000 else 0.01 # step learning rate decay
    for p, grad in zip(parameters, grads):
      #p.data += -lr * p.grad # old way of cheems doge (using PyTorch grad from .backward())
      p.data += -lr * grad # new way of swole doge TODO: enable

    # track stats
    if i % 10000 == 0: # print every once in a while
      print(f'{i:7d}/{max_steps:7d}: {loss.item():.4f}')
    lossi.append(loss.log10().item())

  #   if i >= 100: # TODO: delete early breaking when you're ready to train the full net
  #     break

12297
      0/ 200000: 3.7994
  10000/ 200000: 2.1736
  20000/ 200000: 2.3942
  30000/ 200000: 2.4654
  40000/ 200000: 1.9334
  50000/ 200000: 2.2754
  60000/ 200000: 2.4038
  70000/ 200000: 1.9497
  80000/ 200000: 2.4063
  90000/ 200000: 2.1509
 100000/ 200000: 1.9764
 110000/ 200000: 2.3131
 120000/ 200000: 2.0031
 130000/ 200000: 2.3769
 140000/ 200000: 2.3031
 150000/ 200000: 2.1472
 160000/ 200000: 1.9685
 170000/ 200000: 1.7912
 180000/ 200000: 2.0513
 190000/ 200000: 1.9234


In [56]:
# calibrate the batch norm at the end of training

with torch.no_grad():
  # pass the training set through
  emb = C[Xtrain]
  embcat = emb.view(emb.shape[0], -1)
  hpreact = embcat @ W1 + b1
  # measure the mean/std over the entire training set
  bnmean = hpreact.mean(0, keepdim=True)
  bnvar = hpreact.var(0, keepdim=True, unbiased=True)

In [57]:
# evaluate train and val loss

@torch.no_grad() # this decorator disables gradient tracking
def split_loss(split):
  x,y = {
    'train': (Xtrain, Ytrain),
    'val': (Xdev, Ydev),
    'test': (Xtest, Ytest),
  }[split]
  emb = C[x] # (N, block_size, n_embd)
  embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
  hpreact = embcat @ W1 + b1
  hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
  h = torch.tanh(hpreact) # (N, n_hidden)
  logits = h @ W2 + b2 # (N, vocab_size)
  loss = F.cross_entropy(logits, y)
  print(split, loss.item())

split_loss('train')
split_loss('val')

train 2.0705254077911377
val 2.1104416847229004


In [59]:
# sample from the model
g = torch.Generator().manual_seed(2147483647 + 10)

for _ in range(20):
    
    out = []
    context = [0] * block_size # initialize with all ...
    while True:
      # ------------
      # forward pass:
      # Embedding
      emb = C[torch.tensor([context])] # (1,block_size,d)      
      embcat = emb.view(emb.shape[0], -1) # concat into (N, block_size * n_embd)
      hpreact = embcat @ W1 + b1
      hpreact = bngain * (hpreact - bnmean) * (bnvar + 1e-5)**-0.5 + bnbias
      h = torch.tanh(hpreact) # (N, n_hidden)
      logits = h @ W2 + b2 # (N, vocab_size)
      # ------------
      # Sample
      probs = F.softmax(logits, dim=1)
      ix = torch.multinomial(probs, num_samples=1, generator=g).item()
      context = context[1:] + [ix]
      out.append(ix)
      if ix == 0:
        break
    
    print(''.join(i_to_s[i] for i in out))

mora.
mayah.
seel.
ndhriah.
remmani.
jarlee.
adelyn.
elin.
shi.
jen.
eden.
sana.
arleigh.
malara.
noshubergihimies.
kindreelynn.
novana.
ubrey.
dariyah.
faeh.
